# Generate CMA using tick data service - Python

### Overview
Moving averages smooth the price movements by filtering out the "noise" caused by short-term price fluctuations.They also form the building blocks for a wide range of technical indicators.

Computing moving averages over a defined number of time periods is useful to:
* identify the trend direction
* determine support and resistance levels

This sample demonstrates how to request and plot **on-demand** *cumulative moving average - CMA* from a tick data service.

### Inputs/outputs
Cumulative moving averages' requests require instrument's identifier, date time intervals and number of periods as per inputs.

CMA is the average of all the data points.

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve tick data from the hosted service. The queried endpoint in this script are:
* *TickCmaService *: to directly retrieve tick cma data from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick_analytics.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* as per display package

***

# Run CMA sample

### Step 1: Install packages

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick_analytics.v1.tick_cma_pb2 as tick_cma
import systemathics.apis.services.tick_analytics.v1.tick_cma_pb2_grpc as tick_cma_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Create and process request
To request *CMA* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Cumulative moving average request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'BATS'

#### 3.2 CMA parameters

In [ ]:
field = tick_cma.CMA_PRICE_TRADE; # set the SMA calculation field
sampling = 1 * 60

#### 3.3 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2020, month = 2, day = 18), 
    end_date = date.Date(year = 2020, month = 2, day = 18)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 8, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 20, minutes = 0, seconds = 0)
)

In [ ]:
# generate constraints based on the previous time selection
constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

#### 3.4 CMA request creation
The following code snippets creates *gRPC client*, process *CMA* request and returns the request reply:

In [ ]:
# generate the tick long CMA request
long_request = tick_cma.TickCmaRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    constraints = constraints,
    field = field,
    sampling = duration.Duration(seconds = sampling)
)

In [ ]:
# open a gRPC channel
credentials = grpc.ssl_channel_credentials()
with grpc.secure_channel("apis.systemathics.cloud:443", credentials) as channel:
    
    # instantiate the tick CMA service
    service = tick_cma_service.TickCmaServiceStub(channel)
    
    averages, prices, time_stamps = [], [], []
    # process the tick CMA request
    for c in service.TickCma(request = long_request, metadata = [('authorization', token)]):
        time_stamps.append(c.time_stamp)
        prices.append(c.value)
        averages.append(c.average.value)

The following code snippet displays the total retrieved CMA data points count:

In [ ]:
print('Total CMA data points retrived: {0}'.format(len(averages)))

### Step 4: Visualize data

#### 4.1 Retrieve CMA data

In [ ]:
# create the SMAs dataframe
dates=[datetime.fromtimestamp(ts.seconds) for ts in time_stamps]
d = {'Date': dates, 'Cma': averages, 'Price': prices}
df = pd.DataFrame(data=d)
df

#### 4.4 Plot CMA and prices

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Cma'], name='Cma', marker_color ='orange'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], name='Price', marker_color ='blue'))
fig.update_xaxes(title_text = 'Date')
fig.update_layout(title = 'Tick Cma prices for {0}-{1}'.format(ticker,exchange))
fig.show()